In [1]:
import sys
sys.executable


'/opt/anaconda3/envs/forensic-ai/bin/python'

In [2]:
import yfinance as yf
print("yfinance works")

yfinance works


In [3]:
data = yf.download("RELIANCE.NS", period="5d", interval="5m")
data.head()

/var/folders/wd/40p_1mcx1z5bsp0k55vvm7w40000gn/T/ipykernel_58918/349078462.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("RELIANCE.NS", period="5d", interval="5m")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Datetime,,,,,
2025-12-08 03:45:00+00:00,1545.000000,1551.000000,1541.000000,1541.000000,0
2025-12-08 03:50:00+00:00,1545.599976,1547.900024,1544.000000,1545.300049,86659
2025-12-08 03:55:00+00:00,1544.300049,1546.500000,1544.099976,1546.099976,116518
2025-12-08 04:00:00+00:00,1545.599976,1546.000000,1543.500000,1544.400024,61729
2025-12-08 04:05:00+00:00,1545.500000,1545.599976,1545.099976,1545.500000,38790


In [4]:
import pandas as pd
import numpy as np

# Reset index to make Datetime a column
df = data.reset_index()

# Rename columns for simplicity
df.columns = ["Datetime", "Open", "High", "Low", "Close", "Volume"]

df.head()


,Datetime,Open,High,Low,Close,Volume
0,2025-12-08 03:45:00+00:00,1545.000000,1551.000000,1541.000000,1541.000000,0
1,2025-12-08 03:50:00+00:00,1545.599976,1547.900024,1544.000000,1545.300049,86659
2,2025-12-08 03:55:00+00:00,1544.300049,1546.500000,1544.099976,1546.099976,116518
3,2025-12-08 04:00:00+00:00,1545.599976,1546.000000,1543.500000,1544.400024,61729
4,2025-12-08 04:05:00+00:00,1545.500000,1545.599976,1545.099976,1545.500000,38790


In [5]:
# VWAP calculation
df["VWAP"] = (df["Volume"] * df["Close"]).cumsum() / df["Volume"].cumsum()

# 1️⃣ VWAP deviation (price manipulation signal)
df["vwap_dev"] = abs(df["Close"] - df["VWAP"]) / df["VWAP"]

# 2️⃣ Volume spike ratio (coordination signal)
df["vol_avg"] = df["Volume"].rolling(10).mean()
df["volume_spike_ratio"] = df["Volume"] / df["vol_avg"]

# 3️⃣ Delivery deviation (proxy – explained in report)
df["delivery_dev"] = df["volume_spike_ratio"]

# 4️⃣ Event flag (kept simple for demo)
df["event_flag"] = 0

# Drop NaNs from rolling calculations
df = df.dropna()

df.head()


,Datetime,Open,High,Low,Close,Volume,VWAP,vwap_dev,vol_avg,volume_spike_ratio,delivery_dev,event_flag
9,2025-12-08 04:30:00+00:00,1542.800049,1544.500000,1542.400024,1543.000000,44707,1545.317948,0.001500,55511.0,0.805372,0.805372,0
10,2025-12-08 04:35:00+00:00,1544.199951,1544.599976,1542.599976,1542.900024,38818,1545.159917,0.001463,59392.8,0.653581,0.653581,0
11,2025-12-08 04:40:00+00:00,1548.300049,1548.300049,1543.199951,1544.400024,71689,1545.078074,0.000439,57895.8,1.238242,1.238242,0
12,2025-12-08 04:45:00+00:00,1543.300049,1548.300049,1543.000000,1548.000000,117705,1545.517134,0.001606,58014.5,2.028889,2.028889,0
13,2025-12-08 04:50:00+00:00,1544.500000,1544.800049,1543.000000,1543.000000,24375,1545.441171,0.001580,54279.1,0.449068,0.449068,0


In [9]:
from sklearn.ensemble import IsolationForest

features = [
    "volume_spike_ratio",
    "delivery_dev",
    "vwap_dev",
    "event_flag"
]

X = df[features]

model = IsolationForest(
    n_estimators=100,
    contamination=0.15,
    random_state=42
)

model.fit(X)

df["anomaly_score"] = -model.decision_function(X)
df["is_suspicious"] = model.predict(X)

df[df["is_suspicious"] == -1].head()


,Datetime,Open,High,Low,Close,Volume,VWAP,vwap_dev,vol_avg,volume_spike_ratio,delivery_dev,event_flag,anomaly_score,is_suspicious
29,2025-12-08 06:10:00+00:00,1543.400024,1544.000000,1541.300049,1542.300049,118421,1543.743764,0.000935,57799.7,2.048817,2.048817,0,0.002285,-1
34,2025-12-08 06:35:00+00:00,1541.400024,1543.599976,1540.699951,1543.300049,129249,1543.834851,0.000346,63556.7,2.033601,2.033601,0,0.018531,-1
35,2025-12-08 06:40:00+00:00,1539.800049,1542.000000,1539.800049,1541.400024,471916,1543.365499,0.001273,102770.7,4.591931,4.591931,0,0.265141,-1
36,2025-12-08 06:45:00+00:00,1542.300049,1543.500000,1537.500000,1539.699951,404581,1542.845638,0.002039,138375.2,2.923797,2.923797,0,0.071851,-1
38,2025-12-08 06:55:00+00:00,1540.599976,1543.400024,1538.900024,1543.400024,346846,1542.882331,0.000336,178358.5,1.944656,1.944656,0,0.017712,-1


In [10]:
model_alt = IsolationForest(
    n_estimators=100,
    contamination=0.25,
    random_state=42
)

model_alt.fit(X)
df["is_suspicious_alt"] = model_alt.predict(X)

pd.crosstab(df["is_suspicious"], df["is_suspicious_alt"])


is_suspicious_alt,-1,1
is_suspicious,,
-1,55,0
1,37,274
